# **Tutorial 5** - Advanced AI Agents

In the previous tutorials, you learned how to create AI agents and, more importantly, how to allow them to access the PIRLS data. However, the basic solution presented in tutorial 4 was not sufficient. In this tutorial, we will show you how to improve your solution, enhance your AI agents, and provide general tips for potential improvements to your crew!

# Agenda
1. [How to improve?](#how-to-improve) - recalls the data structure and shows how to make improvements to t he agents.
2. [Modify your code and submit changes](#modify-your-code-and-submit-changes).
3. [Submission status on the website](#submission-status-on-the-website) - shows you how to check the status of yoursubmission during automatic evaluation.
4. [When is your ranking updated?](#when-is-your-ranking-updated) - explains the process of ranking change.
5. [Ways to improve](#ways-to-improve) - shows general tips on how to improve your solution.

# Notebook setup

In [ ]:
pip install -r requirements.txt

# How to improve?
Although the `StudentKnowingCrew` worked for a few questions, it was not sufficient to answer more general questions about PIRLS. This was due to the limited amount of information provided to the agents. To recap, we created this code:

```python
@agent
def database_expert(self) -> Agent:
    return Agent(
        role="Students database expert",
        backstory=dedent("""
            Database expert that knows the structure of PIRLS database regarding students.
            You know that there is the table Students with columns Stident_ID and Country_ID, and a table
            Countries with column Country_ID, Name and Code.
        """),
        goal="Use the tool to query the database and answer the question.",
        llm=self.llm,
        allow_delegation=False,
        verbose=True,
        tools=[
            query_database
        ]
    )
```
We can see that our so-called 'database expert' only knows about two tables from our general PIRLS database: Students and Countries, which is a small section of what we actually have. Referring back to the tutorial on the data, we know that our database structure looks more like this:

[<img src="../images/t2_db_schema.png" width=2000/>](../images/tutorial_2_db_schema.png)

Seeing this clearly shows us how little our current solution understands about the structure of the database. We need a better solution - agents that know much more and can create more advanced queries. Enter the [`DataAnalysisCrew`](./src/submission/crews/basic_crew.py)!

The main difference you’ll notice is that the code is more complex. It now involves 2 agents: the data analyst and the data engineer. Another change is that we now have two static variables that point to 2 configuration YAML files, and the `DataAnalysisCrew` class has a wrapper called `@CrewBase`. These changes allow us to specify configurations for agents and tasks in separate YAML files and help declutter our code. 

Now let's take a look at our new agents. The data analyst has no tools to use. They only create results and analyze the data provided by the data engineer. The data engineer, on the other hand, has three tools and extensive knowledge of the database structure. Their task is to provide data that can help answer the question. Let's see what we can find in the  [agents.yaml](./src/submission/config/agents.yaml) file.

```yaml
lead_data_analyst:
  role: >
    PIRLS lead data analyst
  goal: >
    Answer the research questions using the PIRLS 2021 dataset
  backstory: >
    You are the Lead Data Analyst for the Progress in International Reading Literacy Study (PIRLS) project. 
    Your expertise in data analysis and interpretation is crucial for providing insights into the dataset.
    Your analysis will be used to inform educational policies and practices.
    You focus on questions related to reading literacy and educational outcomes.
   
    While you have a good overview of PIRLS, you always rely on your data engineer to provide you with the necessary data.

# file cut out ...
    
```

This file contains the full configuration for both of our new agents (part for the data enginineer cropped out in the above preview). It's worth noting that the keys for both `lead_data_analyst` and `data_engineer` are the same as the arguments used for agent initialization.

```python
Agent(
  role="...",
  goal="...",
  backstory="..."
)
```

Similarly the same holds true for the [tasks.yaml](./src/submission/config/tasks.yaml) file. The keys used in this file must match the keyword arguments used for initializing the `Task` object. One important thing to notice is that all of the config files can be parametrized. Let's take a look at the tasks configuration file.

```yaml
answer_question_task:
  description: >
    Answer the following question:    
    {user_question}
    
    When applicable, search for relevant data in the PIRLS 2021 dataset.
    
    When answering, always:    
    - Do not comment on topics outside the area of your expertise.     
    - Ensure that your analysis is accurate and relevant to the research questions.
    - Unless instructed otherwise, explain how you come to your conclusions and provide evidence to support your claims.
    - Use markdown format for your final answer.
  expected_output: >
    A clear and concise answer to the question
```

Here we can see that Python string format is being used. In the `description` part we can see the `{user_question}` parameter. How to provide the value for this parameter? You can do that in the `kickoff` method of your crew.

```python
@CrewBase
class DataAnalysisCrew(Submission):
    """Data Analysis Crew for the GDSC project."""
    # Load the files from the config directory
    agents_config = PROJECT_ROOT / 'submission' / 'config' / 'agents.yaml'
    tasks_config = PROJECT_ROOT / 'submission' / 'config' / 'tasks.yaml'

    def __init__(self, llm):
        self.llm = llm

    def run(self, prompt: str) -> str:
        return self.crew().kickoff(inputs={'user_question': prompt})

# ... code cut out
```

We can see that in the `run` method when we call the crew's `kickoff` method we pass an argument called `inputs` which is a `dict` that uses the names of the parameters defined in the YAML file as keys. Although we used only one parameter here, you can define more and pass the values in a similar way.

Another important thing to pay attention to are the tools given to the data engineer agent. They can be found in the [src/submission/tools/database.py](src/submission/tools/database.py). You may already be familiar with one of the functions because it was used by our previous `StudentKnowingCrew`. The other two are quite similar, and their main goal is to reduce the responsibility of generating SQL queries by the AI agents and instead provide them with a straightforward way to use predefined queries. These two tools focus on the questionnaire part of the database. If we take a closer look, we can see that there are a few similar tables, with the main difference being the context of the questions asked in the questionnaires. This can help us abstract some of the queries. Let's take a look at the `get_possible_answers_to_question` tool:

```python
@tool
def get_possible_answers_to_question(
        general_table: Literal['Students', 'Curricula', 'Homes', 'Teachers', 'Schools'],
        questionnaire_answers_table: Literal['StudentQuestionnaireAnswers', 'CurriculumQuestionnaireAnswers', 'HomeQuestionnaireAnswers', 'TeacherQuestionnaireAnswers', 'SchoolQuestionnaireAnswers'],
        questionnaire_entries_table: Literal['StudentQuestionnaireEntries', 'CurriculumQuestionnaireEntries', 'HomeQuestionnaireEntries', 'TeacherQuestionnaireEntries', 'SchoolQuestionnaireEntries'],
        question_code: str
) -> str:
    """Query the database and returns possible answer to a given question

    Args:
        general_table (str): the generic table related to the question topic. Can be one of: 'Students', 'Curricula', 'Homes', 'Teachers', 'Schools'
        questionnaire_answers_table (str): the table related to the `general_table` containing answers.
        questionnaire_entries_table (str): the table related to the `general_table` containing all possible questions.
        question_code (str): the code of the question the full list of possible answers to is returned.

    Returns:
        str: The list of all possible answers to the question with the code given in `question_code`.
    """
    entity_id = 'curriculum_id' if general_table.lower() == 'curricula' else f'{general_table.lower()[:-1]}_id'
    query = f"""
        SELECT DISTINCT ATab.Answer
        FROM {general_table} AS GTab
        JOIN {questionnaire_answers_table} AS ATab ON ATab.{entity_id} = GTab.{entity_id}
        JOIN {questionnaire_entries_table} AS ETab ON ETab.Code = ATab.Code
        WHERE ETab.Code = '{question_code.replace("'", "").replace('"', '')}'
    """

    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return f'Wrong query, encountered exception {e}.'

    ret = ""
    for result in res:
        ret += ", ".join(map(str, result)) + "\n"

    return ret

```

This function is well documented. This is important because the agents use the Python docstrings to determine the purpose of the function. Even if some might say this documentation is a bit too verbose, it will surely help the agents understand how they are supposed to use this tool.

And what does this tool actually do? It returns a list of possible answers that the participants of the PIRLS study had to choose from. For example, the possible answers to the question "About how many books are there in your home?" are as follows (question code ASBG04):

- nan
- Enough to fill two bookcases (101–200 books)
- Enough to fill one shelf (11–25 books)
- None or very few (0–10 books)
- Enough to fill three or more bookcases (more than 200)
- Enough to fill one bookcase (26–100 books)
- Omitted or invalid

To obtain this result the database engineer agent can pass **Students** as the `general_table`, **StudentQuestionnaireAnswers** as the `questionnaire_answers_table`, **StudentQuestionnaireEntries** as the `questionnaire_entries_table`, and **ASBG04** as the `question_code`. That will generate this query:

```sql
select distinct ATab.Answer
from Students as GTab
join StudentQuestionnaireAnswers as ATab on ATab.student_id = GTab.student_id
join StudentQuestionnaireEntries as ETab on ETab.code = ATab.code
where ETab.code = 'ASBG04'

```
This tool might be useful because of the structure of the database. It has 5 questionnaire-related sections that are quite similar to each other, and generating almost the same SQL query repeatedly could be pointless.

# Modify your code and submit changes
If you want to make a new submission to the GDSC, all you have to do is push your desired changes to the **submission** branch of your repository. If the branch is not already there, you can create it. Just remember that the names of the branches that trigger the whole AWS pipeline are: **test_submission** for testing purposes, and **submission** for submitting your new solution.

Now let's submit our new, better crew! Because the code you see was already prepared for you, unless you didn’t create the submission branch, we will have to introduce a dummy change just to be able to push changes and trigger the pipeline. In this case, the submission branch was already created, and I want to submit a new solution, so let's quickly make a minor change in the code. I suggest we modify the src/submission/create_submission.py

[<img src="../images/t5_code_commit_1.png"/>](../images/t5_code_commit_1.png)

Be sure that you are making changes on the **submission** branch!

[<img src="../images/t5_code_commit_2.png"/>](../images/t5_code_commit_2.png)

Fill *author name*, *author eamil* and optionally *commit message* nad press *commit changes*. 

[<img src="../images/t5_code_commit_3.png"/>](../images/t5_code_commit_3.png)

Now you can go to Elastic Container Service (ECS) and see the running task. Go to the clusters and select the gdsc cluster, select the gdscSubmit service, and go to the taks to see its public IP address.

# Submission status on the website

# When is your ranking updated?
The underlying elo ranking for your submission is updated after every battle. It is not directly shown to the participants. What you can see is the leaderboard which is updated **once a day**. The difference between days is calculated based on the *current* and *previous* ranking, which are updated once every hour and they both start at 1500.

# Ways to improve
There are many potential ways to improve the performance of your agents starting from giving them a broader context and better descriptions, using more agents, developing better tools for accessing the database or even adding a completely new ones such as arithmetic tools or tools that can access the internet.

Aditionally you can test how good is your submission in answering the set of questions available on the website. Maybe you can find something that gives your agents a bit of a challenge?